## 0. Introduction

The purpose of this notebook is to explore the Greeks with reference to chapter 6 from *Option Volatility \& Pricing: Advanced Trading Strategies and Techniques* (Natenberg, 1994).

## 1. The Greeks

In mathematical finance, the Greeks are the quantities (known in calculus as partial derivatives; first-order or higher) representing the sensitivity of the price of a derivative instrument such as an option to changes in one or more underlying parameters on which the value of an instrument or portfolio of financial instruments is dependent.

In [1]:
import numpy as np
import scipy.stats as ss

In [2]:
# Define parameters
S0 = 100.0     # spot price
K = 100.0      # strike price
T = 1.0        # time to expiration (in years)
r = 0.04       # annual risk free rate
sigma = 0.2    # annual volatility rate

In [3]:
# Recall from the previous notebook
def closed_formula(S0, K, T, r, sigma, payoff='call'):
    d1 = (np.log(S0 / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if payoff == 'call':
        return S0 * ss.norm.cdf(d1) - K * np.exp(-r * T) * ss.norm.cdf(d2)
    elif payoff == 'put':
        return K * np.exp(-r * T) * ss.norm.cdf(-d2) - S0 * ss.norm.cdf(-d1)

In [4]:
# Calculate option prices using Black Scholes closed formula
call = closed_formula(S0, K, T, r, sigma, 'call')
put = closed_formula(S0, K, T, r, sigma, 'put')

# Print results
print(f"Call price: {call:.3f}")
print(f"Put price: {put:.3f}")

Call price: 9.925
Put price: 6.004


### 1.1 Delta 
Delta, $\Delta$, is the first derivative of the value, $V$, of the option with respect to the underlying asset price, $S$:

$$ \Delta = \frac{\partial V}{\partial S}. $$

For European options priced using the Black-Scholes model, delta is:

$$ \Delta_{\text{call}} = N(d_1), \quad  \quad \Delta_{\text{put}} = N(d_1) - 1. $$

In theory, an option can never gain or lose value more quickly than the underlying, so the delta of a call always has a lower bound of $0.00$ and an upper bound of $1.00$. An option with a delta of $1.00$ will move up or down one full point for each full point move up or down in the price of the underlying (it is moving at $100\%$ of the rate of the underlying). A call with a delta of $0.25$ can be expected to change its value at $25\%$ of the rate of the underlying. If the underlying rises (falls) $1.00$, the option can be expected to rise (fall) $0.25$. 

Puts have characteristics similar to calls except that put values move in the opposite direction of the underlying market. When the underlying rises, puts lose value; when the underlying falls, puts gain value. For this reason, puts always have negative deltas range from $0.00$ for far out-of-the-money puts to $-1.00$ for deeply in-the-money puts. 

If we wish to hedge an option position against the underlying asset, the delta tells us the proper ratio of the position in the underlying asset to options required to establish a neutral hedge. 

There is one other interpretation of the delta which is perhaps os less practical use but is still worth mentioning. The delta is approximately equal to the probability that the option will finish in-the-money. A call with a delta of $0.25$, or a put with a delta of $-0.25$, has approximately a $25\%$ probability of finishing in-the-money. 

In [5]:
def calculate_delta(S0, K, T, r, sigma, payoff='call'):
    d1 = (np.log(S0 / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    if payoff.lower() == 'call':
        return ss.norm.cdf(d1)
    elif payoff.lower() == 'put':
        return ss.norm.cdf(d1) - 1

In [6]:
# Calculate option delta
call_delta = calculate_delta(S0, K, T, r, sigma, payoff='call')
put_delta = calculate_delta(S0, K, T, r, sigma, payoff='put')

# Print results
print(f"Call Delta: {call_delta:.3f}")
print(f"Put Delta: {put_delta:.3f}")

Call Delta: 0.618
Put Delta: -0.382


<center><img src="../figures/delta.png" width="1200" height="400"/></center>

### 1.2 Gamma

Gamma, $\Gamma$, is the rate at which an option's delta changes as the price of the underlying changes:

$$ \Gamma = \frac{\partial \Delta}{\partial S} = \frac{\partial^2 V}{\partial S^2}. $$

For European options priced using the Black-Scholes model, gamma is:

$$ \Gamma = \frac{N'(d_1)}{S_0 \, \sigma \sqrt{T}}. $$

The gamma is usually expressed in deltas gained or lost per one point change in the underlying, with the delta increasing by the amount of the gamma when the underlying rises and falling by the amount of the gamma when the underlying falls.

Regardless of whether we are working with calls or puts, we always add the gamma to the old delta as the underlying rises and subtract the gamma from the old delta as the underlying falls (follows intuition). 

When a trader is long options, whether calls or puts, he has a long gamma position. When he is short options, he has a short gamma position. A large gamma number indicates a high degree of risk (and vice versa). 

Every option trader needs to look not only at current directional risk (the delta), but also at how that directional risk will change if the underlying market begins to move (the gamma). Gamma is greatest for an option which is at-the-money and becomes progressively smaller as an option moves either into- or out-of-the-money. 

In [7]:
def calculate_gamma(S0, K, T, r, sigma):
    d1 = (np.log(S0 / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    return ss.norm.pdf(d1) / (S0 * sigma * np.sqrt(T))

In [8]:
# Calculate option gamma
call_gamma = calculate_gamma(S0, K, T, r, sigma)
put_gamma = calculate_gamma(S0, K, T, r, sigma)

# Print results
print(f"Call Gamma: {call_gamma:.3f}")
print(f"Put Gamma: {put_gamma:.3f}")

Call Gamma: 0.019
Put Gamma: 0.019


<center><img src="../figures/gamma.png" width="950" height="215"/></center>

### 1.3 Theta
Theta, $\Theta$, is the rate at which an option loses value as time passes:

$$ \Theta = - \frac{\partial V}{\partial \tau}. $$

For European options priced using the Black-Scholes model, theta is:

$$ \Theta_{call} = - \frac{S_0 \, N'(d_1) \sigma}{2 \sqrt{T}} - r \, K e^{-r T} N(d_2), \quad \quad
   \Theta_{put} = - \frac{S_0 \, N'(d_1) \sigma}{2 \sqrt{T}} + r \, K e^{-r T} N(-d_2). $$

Note that theta here is calculated in years, but if we divide by 365 we get theta in days. It is usually expressed in points lost per day, when all other conditions remain the same. An option with a theta of $0.05$ will lose $0.05$ in value for each day that passes with no change in other market conditions. If the option is woth $2.75$ today, then tomorrow it will be worth $2.70$, the day after that it will be worth $2.65$.

To remind us that time runs in only one direction, the theta written as a negative number and represents a loss in the option's value as time passes. 

As a general principle, an option position will have gamma and theta of opposite signs. Moreover, the relative size of the gamma and theta will correlate. Every option position is a tradeoff between market movement and time decay. If price movement in the underlying contract will help a trader (positive gamma), the passage of time will hurt (negative theta). And vice versa.

In [9]:
def calculate_theta(S0, K, T, r, sigma, payoff='call'):
    d1 = (np.log(S0 / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if payoff == 'call':
        return (-S0 * ss.norm.pdf(d1) * sigma / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * ss.norm.cdf(d2)) / 365.0
    elif payoff == 'put':
        return (-S0 * ss.norm.pdf(d1) * sigma / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * ss.norm.cdf(-d2)) / 365.0

In [10]:
# Calculate option theta
call_theta = calculate_theta(S0, K, T, r, sigma, payoff='call')
put_theta = calculate_theta(S0, K, T, r, sigma, payoff='put')

# Print results
print(f"Call Theta: {call_theta:.3f}")
print(f"Put Theta: {put_theta:.3f}")

Call Theta: -0.016
Put Theta: -0.006


<center><img src="../figures/theta.png" width="600" height="380"/></center>

### 1.4 Vega

Vega, $\mathcal{V}$, is the derivative of the option value with respect to the volatility of the underlying asset price:

$$ \mathcal{V} = \frac{\partial V}{\partial \sigma}. $$

For European options priced using the Black-Scholes model, vega is:

$$ \mathcal{V} = S_0 \sqrt{T} \, N'(d_1) $$

Since all options gain value with rising volatility, the vega for both calls and puts is positive. If an option has a vega of $0.15$, for each percentage point increase (decrease) in volatility, the option will gain (lose) $0.15$ in value. If the option has a theoretical value of $3.25$ at a volatility of $20\%$, then it will have a theoretical value of $3.40$ at a volatility of $21\%$, and a theoretical value of $3.10$ at a volatility of $19\%$.

In [11]:
def calculate_vega(S0, K, T, r, sigma):
    d1 = (np.log(S0 / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    return (S0 * np.sqrt(T) * ss.norm.pdf(d1)) / 100

In [12]:
# Calculate option vega
call_vega = calculate_vega(S0, K, T, r, sigma)
put_vega = calculate_vega(S0, K, T, r, sigma)

# Print results
print(f"Call Vega: {call_vega:.3f}")
print(f"Put Vega: {put_vega:.3f}")

Call Vega: 0.381
Put Vega: 0.381


<center><img src="../figures/vega.png" width="600" height="380"/></center>

### 1.5 Rho

Rho, $\rho$, measures the sensitivity of an option's value to a change in interest rates:

$$ \rho = \frac{\partial V}{\partial r}. $$ 

For European options priced using the Black-Scholes model, rho is:

$$ \rho_{call} = K T e^{-r T} N(d_2), \quad \quad \rho_{put} = -K T e^{-r T} N (-d_2). $$

Unlike the other greeks, one cannot generalilze about the rho since its characteristics depend on the type of underlying instrument and the settlement procedure for the options. 

In general, it is best to think of a call as a substitute for purchasing the underlying stock, and a put as a substitute for selling the underlying stock. If interest rates are high, we will prefer the call instead of purchasing the asset since the outright purchase requires a much larger cash outlay, and therefore a greater carrying cost. Hence, rising interest rates cause stock option calls to rise in value (and vice versa).

In a high interest rate environment, the sale of stock will be more desirable than purchasing a put since the cash credit resulting from the sale will earn greater interest. Hence, rising interest rates cause stock option puts to fall (and vice versa).

In [13]:
def calculate_rho(S0, K, T, r, sigma, payoff='call'):
    d1 = (np.log(S0 / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if payoff == 'call':
        return (K * T * np.exp(-r * T) * ss.norm.cdf(d2)) / 100
    elif payoff == 'put':
        return (-K * T * np.exp(-r * T) * ss.norm.cdf(-d2)) / 100

In [14]:
# Calculate option rho
call_rho = calculate_rho(S0, K, T, r, sigma, payoff='call')
put_rho = calculate_rho(S0, K, T, r, sigma, payoff='put')

# Print results
print(f"Call Rho: {call_rho:.3f}")
print(f"Put Rho: {put_rho:.3f}")

Call Rho: 0.519
Put Rho: -0.442


<center><img src="../figures/rho.png" width="600" height="380"/></center>